Задание 1

Используйте файл с оценками фильмов ml-latest-small/ratings.csv. Посчитайте среднее время жизни пользователей, которые выставили более 100 оценок. Под временем жизни понимается разница между максимальным и минимальным значением столбца timestamp для данного значения userId.

In [5]:
import pandas as pd

In [10]:
ratings = pd.read_csv('ratings.csv')
ratings.head()

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


In [62]:
def time_user(user_data):
    max_timestamp = max([int(x[3]) for x in user_data])
    min_timestamp = min([int(x[3]) for x in user_data])
    return  max_timestamp - min_timestamp

In [73]:
import datetime

def convert(n):
    return str(datetime.timedelta(seconds = n))

In [74]:
previous_user = None
user_data = []
mean_list = []

with open('ratings.csv') as f:
    f.readline()
    for line in f:
        userId, movieId, rating, timestamp = line.strip().split(',')
        #print(userId, movieId, rating, timestamp)
        if previous_user:
            if userId == previous_user:
                user_data.append([userId, movieId, rating, timestamp])
            else:
                mean_list.append(time_user(user_data))
                user_data = [[userId, movieId, rating, timestamp]]
                previous_user = userId
        else:
            previous_user = userId
            user_data.append([userId, movieId, rating, timestamp])
mean_list.append(time_user(user_data))
mean_value = sum(mean_list)/len(mean_list)

print(convert(mean_value))

        

201 days, 2:01:01.004471


Задание 2

Дана статистика услуг перевозок клиентов компании по типам (см. файл с кодом занятия). Необходимо сформировать две таблицы:
таблицу с тремя типами выручки для каждого client_id без указания адреса клиента
аналогичную таблицу по типам выручки с указанием адреса клиента
Обратите внимание, что в процессе объединения таблиц данные не должны теряться.

In [6]:
rzd = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115],
        'rzd_revenue': [1093, 2810, 10283, 5774, 981]
    }
)
rzd

,client_id,rzd_revenue
0,111,1093
1,112,2810
2,113,10283
3,114,5774
4,115,981


In [7]:
auto = pd.DataFrame(
    {
        'client_id': [113, 114, 115, 116, 117],
        'auto_revenue': [57483, 83, 912, 4834, 98]
    }
)
auto

,client_id,auto_revenue
0,113,57483
1,114,83
2,115,912
3,116,4834
4,117,98


In [8]:
air = pd.DataFrame(
    {
        'client_id': [115, 116, 117, 118],
        'air_revenue': [81, 4, 13, 173]
    }
)
air

,client_id,air_revenue
0,115,81
1,116,4
2,117,13
3,118,173


In [9]:
client_base = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115, 116, 117, 118],
        'address': ['Комсомольская 4', 'Энтузиастов 8а', 'Левобережная 1а', 'Мира 14', 'ЗЖБИиДК 1', 
                    'Строителей 18', 'Панфиловская 33', 'Мастеркова 4']
    }
)
client_base

,client_id,address
0,111,Комсомольская 4
1,112,Энтузиастов 8а
2,113,Левобережная 1а
3,114,Мира 14
4,115,ЗЖБИиДК 1
5,116,Строителей 18
6,117,Панфиловская 33
7,118,Мастеркова 4


In [15]:
revenue_rzd_air_client_base = (rzd
                               .merge(auto, on = 'client_id', how = 'outer')
                               .merge(air, on = 'client_id', how = 'outer'))
revenue_rzd_air_client_base

,client_id,rzd_revenue,auto_revenue,air_revenue
0,111,1093.0,NaN,NaN
1,112,2810.0,NaN,NaN
2,113,10283.0,57483.0,NaN
3,114,5774.0,83.0,NaN
4,115,981.0,912.0,81.0
5,116,NaN,4834.0,4.0
6,117,NaN,98.0,13.0
7,118,NaN,NaN,173.0


In [16]:
revenue_rzd_air_client_base_address = revenue_rzd_air_client_base.merge(client_base, on = 'client_id', how = 'outer')
revenue_rzd_air_client_base_address

,client_id,rzd_revenue,auto_revenue,air_revenue,address
0,111,1093.0,NaN,NaN,Комсомольская 4
1,112,2810.0,NaN,NaN,Энтузиастов 8а
2,113,10283.0,57483.0,NaN,Левобережная 1а
3,114,5774.0,83.0,NaN,Мира 14
4,115,981.0,912.0,81.0,ЗЖБИиДК 1
5,116,NaN,4834.0,4.0,Строителей 18
6,117,NaN,98.0,13.0,Панфиловская 33
7,118,NaN,NaN,173.0,Мастеркова 4


Задание 3

В задаче сквозной аналитики вам предоставили данные по местоположению пользователей. Т. е. для каждого user_id известна последовательность координат (широта/долгота), когда они требовались приложению для полноценной работы. Как бы вы добавили эти сведения в таблицу визитов и покупок? Для составления ответа можно использовать вопросы:
У каждого пользователя известен набор координат. А для связывания с визитом или фактом покупки скорее всего потребуется одно-два числа. Как их получить?
Наборы координат одного и того же пользователя могут быть значительно удалены друг от друга. Как это отразится на вопросе расчетах пункта 1?
Какие дополнительные признаки можно получить из координат? Ведь это просто числа, которые сами по себе мало что дают.

1) для получения этих чисел мы можем сделать проверку, купил пользователь(1) или просто посетил сайт(0), далее можем соеделить их с геолокацией
2) скорее всего при первом визите это первая координата, при покупке вторая координата. можем посмотреть расстояние, или как место повлияла на покупку
3) можно узнать в каком городе или стране чаще всего используется сайт, или в каком месте отдельный пользователь чаще всего делает покупки и с помощью этих данных оптимизировать рекламу 